In [0]:
from pyspark.sql.functions import col, current_date, year, month, dayofmonth, when, date_format, to_date, regexp_replace, regexp_extract
from delta.tables import *

# Leer tablas Bronze

# Dataframes
players_bronze = (
    spark
    .table("football.`bronze-football-data`.players_bronze"))
games_bronze = (
    spark
    .table("football.`bronze-football-data`.games_bronze"))
clubs_bronze = (
    spark
    .table("football.`bronze-football-data`.clubs_bronze"))
competitions_bronze = (
    spark
    .table("football.`bronze-football-data`.competitions_bronze"))

#Delta Tables
game_lineups_bronze_dt = (
    DeltaTable
    .forName(spark, "football.`bronze-football-data`.game_lineups_bronze"))
game_events_bronze_dt = (
    DeltaTable
    .forName(spark, "football.`bronze-football-data`.game_events_bronze"))
appearances_bronze_dt = (
    DeltaTable
    .forName(spark, "football.`bronze-football-data`.appearances_bronze"))
club_games_bronze_dt = (
    DeltaTable
    .forName(spark, "football.`bronze-football-data`.club_games_bronze"))
player_valuations_bronze_dt = (
    DeltaTable
    .forName(spark, "football.`bronze-football-data`.player_valuations_bronze"))
transfers_bronze_dt = (
    DeltaTable
    .forName(spark, "football.`bronze-football-data`.transfers_bronze"))


# LIMPIAR TABLA players
players_silver = (
    players_bronze
    .dropDuplicates()
    #Eliminar columnas no necesarias
    .drop("url")
)


# LIMPIAR TABLA games
games_silver = (
    games_bronze
    .dropDuplicates()
    .drop("url")
    .filter(col("date").isNotNull())
    .withColumn(
    #Limpiar datos columna round    
        "round", 
        when(
            col("competition_type") == "domestic_league",
            regexp_replace(col("round"), r"\. Matchday", "")
            ).otherwise(col("round"))
        )
)

#CLONAR TABLA game_lineups
(game_lineups_bronze_dt
    .clone(
        target="football.`silver-football-data`.game_lineups_silver", 
        isShallow=True,
        replace=True))

#CLONAR TABLA game_events
(game_events_bronze_dt
    .clone(
        target="football.`silver-football-data`.game_events_silver", 
        isShallow=True,
        replace=True))


#CLONAR TABLA appearances 
(appearances_bronze_dt
    .clone(
        target="football.`silver-football-data`.appearances_silver", 
        isShallow=True, 
        replace=True))

#CLONAR TABLA clubs_games
(club_games_bronze_dt
    .clone(
        target="football.`silver-football-data`.club_games_silver", 
        isShallow=True, 
        replace=True))

#LIMPIAR TABLA clubs
clubs_silver = (
    clubs_bronze
    .dropDuplicates()
    .drop("club_code","foreigners_percentage","total_market_value", "net_transfer_record", "coach_name", "url", "filename")
)

#LIMPIAR TABLA competitions
competitions_silver = (
    competitions_bronze
    .dropDuplicates()
    .drop("competition_code", "country_id", "domestic_league_code","url")
)

#CLONAR TABLA player_valuations
(player_valuations_bronze_dt
    .clone(
        target="football.`silver-football-data`.player_valuations_silver",
        isShallow=True, 
        replace=True))

#CLONAR TABLA transfers
(transfers_bronze_dt
    .clone(
        target="football.`silver-football-data`.transfers_silver", 
        isShallow=True, 
        replace=True))
    

# Guardar como tablas Silver
(players_silver
    .write
    .format("delta")
    .option("overwriteSchema", "true")        
    .mode("overwrite")
    .saveAsTable("football.`silver-football-data`.players_silver"))
(games_silver
    .write
    .format("delta")
    .option("overwriteSchema", "true")        
    .mode("overwrite")
    .saveAsTable("football.`silver-football-data`.games_silver"))
(clubs_silver
    .write
    .format("delta")        
    .option("overwriteSchema", "true")
    .mode("overwrite")
    .saveAsTable("football.`silver-football-data`.clubs_silver"))
(competitions_silver
    .write
    .format("delta")
    .option("overwriteSchema", "true")
    .mode("overwrite")
    .saveAsTable("football.`silver-football-data`.competitions_silver"))
